<div class="table-of-contents" style="background-color:#E8E4E3; padding: 20px; margin: 10px; font-size: 110%; border-radius: 25px; box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);">
  <h1 style="color:#000000;">TOC</h1>
  <ol>
     <li><a href="#2" style="color: #000000;">1. Imports</a></li>
    <li><a href="#4" style="color: #000000;">2. Data Preprocessing</a></li>
    <li><a href="#5" style="color: #000000;">3. Model </a></li>
    <li><a href="#6" style="color: #000000;">4. Evaluation</a></li>
    <li><a href="#7" style="color: #000000;">5. Conclusion</a></li>
    <li><a href="#8" style="color: #000000;">6. Thank You</a></li>  
  </ol>
</div>

<a id="1"></a>
<h1 style='background:#E8E4E3;border:0; color:000000;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #000000; text-align: center;'>Imports</center></h1>


# Imports
       

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import plotly.express as px

from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Conv2D,Add,MaxPooling2D, Dense, BatchNormalization,Input,Flatten, Dropout,GlobalMaxPooling2D,Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

<a id="2"></a>
<h1 style='background:#E8E4E3;border:0; color:000000;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #000000;'>Data Preprocessing</center></h1>


       
    
# Data Preprocessing

In [2]:
os.listdir('/kaggle/input/100-bird-species')

['info.txt',
 'MobileNet-475-(224 X 224)-98.85.h5',
 'images to predict',
 'valid',
 'test',
 'birds.csv',
 'train']

**First let's define path to train, test, validation**

In [3]:
train = '/kaggle/input/100-bird-species/train'
validation = '/kaggle/input/100-bird-species/valid'
test = '/kaggle/input/100-bird-species/test'

## Create data generators

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255)#initialize train generator 
                                 


valid_datagen = ImageDataGenerator(rescale = 1.0/255.) #initialize validation generator 

test_datagen = ImageDataGenerator(rescale = 1.0/255.) #initialize validation generator 

In [5]:
train_generator = train_datagen.flow_from_directory(train, target_size=(224,224),batch_size=32,class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory(validation, target_size=(224,224),batch_size=32,class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test, target_size=(224,224),batch_size=32,class_mode='categorical')

Found 75394 images belonging to 475 classes.
Found 2375 images belonging to 475 classes.
Found 2375 images belonging to 475 classes.


<a id="3"></a>
<h1 style='background:#E8E4E3;border:0; color:000000;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #000000;'>Model</center></h1>


       
    
# Model


**As dicussed in this [notebook](https://www.kaggle.com/code/fareselmenshawii/introdution-to-transfer-learning) Transfer Learning can give us a  head start so**


**We will be using InceptionV3 with fine tuning**

In [6]:
#Load Model
inception = tf.keras.applications.InceptionV3(weights='imagenet',include_top=False,input_shape=(224,224,3))


2023-02-07 21:29:25.582170: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-07 21:29:25.585994: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-07 21:29:25.586687: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-07 21:29:25.587852: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

87924736/87910968 [==============================] - 4s 0us/step


In [7]:
inception.trainable = True
for layer in inception.layers[:197]:
    layer.trainable = False 

In [8]:
#get the last layer    
last_layer = inception.get_layer('mixed7')

print('last layer output shape: ', last_layer.output_shape)

layer_output = last_layer.output

last layer output shape:  (None, 12, 12, 768)


**Now let's add our network**

In [9]:
n_categories = len(os.listdir('/kaggle/input/100-bird-species/train'))# number of categories
print(n_categories)

475


In [10]:
# x  = BatchNormalization()(layer_output)
x = Flatten()(layer_output)
x = Dense(1024,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = Dropout(0.4)(x)
x = Dense(n_categories, activation='softmax')(x)

model = Model(inputs=inception.inputs, outputs=x)

In [11]:
# Set the training parameters
model.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

In [12]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
    
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [13]:
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            epochs = 25,
callbacks=[callback])

2023-02-07 21:29:35.112380: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/25


2023-02-07 21:29:40.090094: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


2357/2357 [==============================] - 439s 182ms/step - loss: 3.1015 - accuracy: 0.4903 - val_loss: 1.0761 - val_accuracy: 0.8867
Epoch 2/25
2357/2357 [==============================] - 245s 104ms/step - loss: 1.2090 - accuracy: 0.8363 - val_loss: 0.8432 - val_accuracy: 0.9175
Epoch 3/25
2357/2357 [==============================] - 245s 104ms/step - loss: 0.8720 - accuracy: 0.8946 - val_loss: 0.7262 - val_accuracy: 0.9297
Epoch 4/25
2357/2357 [==============================] - 252s 107ms/step - loss: 0.7286 - accuracy: 0.9137 - val_loss: 0.6688 - val_accuracy: 0.9335
Epoch 5/25
2357/2357 [==============================] - 256s 109ms/step - loss: 0.6486 - accuracy: 0.9230 - val_loss: 0.6186 - val_accuracy: 0.9309
Epoch 6/25
2357/2357 [==============================] - 247s 105ms/step - loss: 0.5917 - accuracy: 0.9291 - val_loss: 0.5769 - val_accuracy: 0.9368
Epoch 7/25
2357/2357 [==============================] - 254s 108ms/step - loss: 0.5441 - accuracy: 0.9341 - val_loss: 0.559

<a id="4"></a>
<h1 style='background:#E8E4E3;border:0; color:000000;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #000000;'>Evaluation</center></h1>


       
    
# Evaluation

In [14]:
results = pd.DataFrame(history.history)
results.tail()

,loss,accuracy,val_loss,val_accuracy,lr
20,0.124070,0.987201,0.322491,0.936421,0.000033
21,0.112971,0.988832,0.328272,0.936421,0.000030
22,0.103810,0.990145,0.312035,0.939368,0.000027
23,0.095902,0.991286,0.303739,0.941053,0.000025
24,0.092094,0.991617,0.309759,0.939368,0.000022


In [15]:
fig = px.line(results,y=[results['accuracy'],results['val_accuracy']],template="seaborn",color_discrete_sequence=['#7F00FF','#00bfff'])
fig.update_layout(   
    title_font_color="#41BEE9", 
    xaxis=dict(color="#41BEE9",title='Epochs'), 
    yaxis=dict(color="#41BEE9")
 )
fig.show()

In [16]:
fig = px.line(results,y=[results['loss'],results['val_loss']],template="seaborn",color_discrete_sequence=['#7F00FF','#00bfff'])
fig.update_layout(   
    title_font_color="#41BEE9", 
    xaxis=dict(color="#41BEE9",title='Epochs'), 
    yaxis=dict(color="#41BEE9")
 )
fig.show()

In [17]:
model.save('model.h5')

In [18]:
model.evaluate(test_generator,verbose=0)

[0.18761464953422546, 0.9679999947547913]

<a id="6"></a>
<h1 style='background:#E8E4E3;border:0; color:000000;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #000000;'>Conclusion</center></h1>


       
    
# Conclusion

**We were able to get decent results with fine-tuning and learning rate sheduler**

**After trying data augmentation many times it seems to do worse with the current architecure**

<a id="6"></a>
<h1 style='background:#E8E4E3;border:0; color:000000;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '><center style='color: #000000;'>Thank You</center></h1>


       
    
# Thank You
**Thank you for taking your time and going through this notebook**

**If you have any suggestions please let me know**